In [1]:
from collections import defaultdict

In [2]:
from itertools import islice
with open("wikicorpus_01") as myfile:
    head = list(islice(myfile, 651))

In [3]:
sent = []
for s in head:
    sent.append(s.split())

In [4]:
words = []
sentences = []
for s in sent:
    if len(s) == 4 and s[0] not in {'(', ')', ','} and '[' not in s[1]:
        words.append((s[0], s[1], s[2], s[3]))
        #if 'endofarticle' in s[1]:
        #    sentences.append(words)
        #    words = []

In [5]:
words[:4]

[('Néstor_Gabriel_Martinena', 'néstor_gabriel_martinena', 'NP00000', '0'),
 ('La', 'el', 'DA0FS0', '0'),
 ('Plata', 'plata', 'NP00000', '10501473'),
 ('Buenos_Aires', 'buenos_aires', 'NP00000', '0')]

In [6]:
count = dict()
for fst,snd,trd, fth in words:
    if fst not in count:
        count[fst] = 1
    else:
        count[fst] += 1

In [7]:
aux = []
check = []
for fst,snd,trd, fth in words:
    aux.append((fst,snd,trd, fth))
    if fst == '.':
        check.append(aux)
        aux = []

In [8]:
check[:2]

[[('Néstor_Gabriel_Martinena', 'néstor_gabriel_martinena', 'NP00000', '0'),
  ('La', 'el', 'DA0FS0', '0'),
  ('Plata', 'plata', 'NP00000', '10501473'),
  ('Buenos_Aires', 'buenos_aires', 'NP00000', '0'),
  ('Argentina', 'argentina', 'NP00000', '02035234'),
  ('es', 'ser', 'VSIP3S0', '01775973'),
  ('un', 'uno', 'DI0MS0', '0'),
  ('futbolista', 'futbolista', 'NCCS000', '0'),
  ('argentino', 'argentino', 'AQ0MS0', '00278090'),
  ('.', '.', 'Fp', '0')],
 [('Juega', 'jugar', 'VMM02S0', '00727813'),
  ('de', 'de', 'SPS00', '0'),
  ('delantero', 'delantero', 'NCMS000', '00466114'),
  ('y', 'y', 'CC', '0'),
  ('su', 'su', 'DP3CS0', '0'),
  ('equipo', 'equipo', 'NCMS000', '06093198'),
  ('actual', 'actual', 'AQ0CS0', '01667781'),
  ('es', 'ser', 'VSIP3S0', '01775973'),
  ('Gimnasia', 'gimnasia', 'NP00000', '00275488'),
  ('y', 'y', 'CC', '0'),
  ('Esgrima_La_Plata_de_la_Primera_División_de_Argentina',
   'esgrima_la_plata_de_la_primera_división_de_argentina',
   'NP00000',
   '0'),
  ('.', '.'

In [9]:
dicc_backward = defaultdict(list)
dicc_forward = defaultdict(list)
for s in check:
    for i,w in enumerate(s):
        if i != 0:
            dicc_backward[w[0]].append((s[i-1][0], s[i-1][1], s[i-1][2], s[i-1][3]))
        else:
            dicc_backward[w[0]].append(('[W.Start]', '[L.Start]', '[T.Start]', '[S.Start]'))
        if i != len(s)-1:
            dicc_forward[w[0]].append((s[i+1][0], s[i+1][1], s[i+1][2], s[i+1][3]))
        else:
            dicc_forward[w[0]].append(('[W.End]', '[T.End]', '[T.End]', '[S.End]'))

In [10]:
dicc_backward['equipo']

[('su', 'su', 'DP3CS0', '0'),
 ('el', 'el', 'DA0MS0', '0'),
 ('su', 'su', 'DP3CS0', '0'),
 ('el', 'el', 'DA0MS0', '0')]

In [11]:
dicc_forward['equipo']

[('actual', 'actual', 'AQ0CS0', '01667781'),
 ('Racing_Club', 'racing_club', 'NP00000', '0'),
 ('y', 'y', 'CC', '0'),
 ('de', 'de', 'SPS00', '0')]

In [12]:
count['La']

1

In [29]:
dicc = dict()
for fst,snd,trd,fth in words:
    features = defaultdict(int)
    pos = 'PoS__' + snd
    if snd not in dicc and snd not in {'.', ',', ':', ';'} and snd.isalpha(): #and count[fst] > 2:
        dicc[snd] = features
        features[pos] = 1
        features[fth] = 1 #synset
        for i in dicc_backward[fst]:
            features[i[1]+"-1"] += 1
            features[i[2]+"-1"] += 1
            features[i[3]+"-1"] += 1
        for i in dicc_forward[fst]:
            features[i[1]+"+1"] += 1
            features[i[2]+"+1"] += 1
            features[i[3]+"+1"] += 1
        
    elif fst in dicc:
        has_it = dicc[fst]
        if fth in has_it:
            has_it[fth] += 1
        else:
            has_it[fth] = 1
        if pos in has_it:
            has_it[pos] += 1
        else:
            has_it[pos] = 1

In [30]:
dicc['cuyo']

defaultdict(int,
            {'0': 3,
             '0-1': 3,
             '04538868+1': 1,
             '06239374+1': 1,
             '07076177+1': 1,
             'NCMS000+1': 3,
             'NP00000-1': 3,
             'PoS__cuyo': 3,
             'ayuntamiento_de_valle_de_losa-1': 1,
             'ha-1': 1,
             'la_zarza-1': 1,
             'plan+1': 1,
             'promotor+1': 1,
             'sistema+1': 1})

In [31]:
lista_para_vectorizar = []
check_list = []
for key, value in dicc.items():
    check_list.append(key)
    lista_para_vectorizar.append(value)

In [32]:
from sklearn.feature_extraction import DictVectorizer
v = DictVectorizer(sparse=False)
X = v.fit_transform(lista_para_vectorizar)

In [33]:
from sklearn.decomposition import TruncatedSVD

#Reducción de dimensionalidad
svd = TruncatedSVD(n_components=47, n_iter=7, random_state=42)
svd.fit_transform(X)

array([[  2.41147769e+01,   3.54558162e+01,  -8.65994619e+00, ...,
          7.07418848e-03,  -1.08162165e-02,   1.10827980e-02],
       [  7.58197799e-01,  -3.55861594e-01,   7.61374223e-01, ...,
          3.47463084e-01,   2.64380631e-02,   1.30348104e-01],
       [  2.61404266e-01,   2.20514612e-02,   5.85892450e-01, ...,
         -2.18886512e-01,  -1.91572363e-01,   1.94636976e-01],
       ..., 
       [  1.22247354e+00,   1.32614278e-01,   4.13726265e-01, ...,
         -4.45731712e-02,   2.11204511e-01,  -2.80679038e-01],
       [  6.41333078e-01,  -3.06928154e-01,   8.84307091e-01, ...,
          1.45749191e-02,  -1.94442750e-03,   2.32131945e-02],
       [  4.42536786e-01,  -2.59694736e-01,   4.60542666e-01, ...,
          2.14797830e-03,  -2.23666581e-02,   5.29656100e-02]])

In [34]:
import sklearn
Y = sklearn.preprocessing.normalize(X, axis=0)

In [21]:
Y

array([[ 0.        ,  0.        ,  0.09996876, ...,  0.        ,
         0.70710678,  0.        ],
       [ 0.        ,  0.7148096 ,  0.79975012, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.29242211,  0.17494534, ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  0.04873702,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.02499219, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.89442719,  0.        ,  0.07497657, ...,  0.        ,
         0.        ,  0.        ]])

In [35]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters=5)
km.fit(Y)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [36]:
labels = km.predict(Y)

In [27]:
print(labels)

[0 1 2 0 0 4 3 0 0 0 0 0 0 0 0 0 0 0 0]


In [28]:
clusters = defaultdict(set)
for i, label in enumerate(labels):
    clusters[label].add(check_list[i])
print(clusters)

defaultdict(<class 'set'>, {0: {'partido', 'su', 'uno', 'con', 'localidad', 'tiempo', 'altura', 'planta', 'por', 'equipo', 'minuto', 'que', 'cuyo', 'a', 'ser'}, 1: {'de'}, 2: {'y'}, 4: {'en'}, 3: {'el'}})
